In [1]:
import pandas as pd
import numpy as np

In [2]:
x2_raw = pd.read_csv('../data/X2.csv')
y2_raw = pd.read_csv('../data/Y2.csv')

*Connected components in graph code (below) - taken from geeks for geeks.*

In [3]:
class Graph:
 
    # init function to declare class variables
    def __init__(self, V):
        self.V = V
        self.adj = [[] for i in range(V)]
 
    def DFSUtil(self, temp, v, visited):
 
        # Mark the current vertex as visited
        visited[v] = True
 
        # Store the vertex to list
        temp.append(v)
 
        # Repeat for all vertices adjacent
        # to this vertex v
        for i in self.adj[v]:
            if visited[i] == False:
 
                # Update the list
                temp = self.DFSUtil(temp, i, visited)
        return temp
 
    # method to add an undirected edge
    def addEdge(self, v, w):
        self.adj[v].append(w)
        self.adj[w].append(v)
 
    # Method to retrieve connected components
    # in an undirected graph
    def connectedComponents(self):
        visited = []
        cc = []
        for i in range(self.V):
            visited.append(False)
        for v in range(self.V):
            if visited[v] == False:
                temp = []
                cc.append(self.DFSUtil(temp, v, visited))
        return cc

In [4]:
matches = y2_raw[y2_raw['label'] == 1].iloc[:, :2]

In [7]:
# d maps instance id -> unique integers
d = {}
for i in range(len(matches)):
    lid = matches.iloc[i, 0]
    rid = matches.iloc[i, 1]
    if lid not in d:
        d[lid] = len(d)
    
    if rid not in d:
        d[rid] = len(d)

# dr is reverse mapping for d
dr = {v:k for k,v in d.items()}

In [10]:
g = Graph(len(d))

for i in range(len(matches)):
    lid = matches.iloc[i, 0]
    rid = matches.iloc[i, 1]
    g.addEdge(d[lid], d[rid])

cc = g.connectedComponents()   

In [11]:
grouped_data = []
for c in cc:
    for n in c:
        grouped_data.append(x2_raw[x2_raw["instance_id"] == dr[n]].values[0])
    grouped_data.append(["-----"]*len(grouped_data[0]))

In [12]:
new_df = pd.DataFrame(grouped_data, columns=x2_raw.columns)
new_df.to_csv('../data/clustered_x2.csv', index=False)